(In order to load the stylesheet of this notebook, execute the last code cell in this notebook)

# Analyzing Hotel Ratings on Tripadvisor

In this homework we will focus on practicing two techniques: web scraping and regression. For the first part, we will get some basic information for each hotel in Boston. Then, we will fit a regression model on this information and try to analyze it.

** Task 1 (30 pts)**

We will scrape the data using Beautiful Soup. For each hotel that our search returns, we will get the information below.

![Information to be scraped](hotel_info.png)

Of course, feel free to collect even more data if you want. 

In [1]:
import urllib2
import csv
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
url_list = []
page = urllib2.urlopen('https://www.tripadvisor.com/Hotels-g60745-Boston_Massachusetts-Hotels.html').read()
soup = BeautifulSoup(page, "lxml")
list = soup.findAll('div', attrs = {'class':'prw_rup prw_common_short_cell_thumbnail'})
myNext = soup.findAll('a', attrs = {'class':'nav next ui_button primary taLnk'})
for s in list:
    tmp = BeautifulSoup(str(s),"lxml")
    tmpList = tmp.findAll('a', href=True)
    for element in tmpList:
        arr = str(element).split(' ')
        tmp_arr = str(arr[1]).split('=')
        url = str(tmp_arr[1])[2:-1]
        url_list.append(url)
        
while len(myNext) == 1:
    nextTmp = myNext[0]
    st = nextTmp.get('href')
    url = 'https://www.tripadvisor.com' + st
    page = urllib2.urlopen(url).read()
    soup = BeautifulSoup(page, "lxml")
    list = soup.findAll('div', attrs = {'class':'prw_rup prw_common_short_cell_thumbnail'})
    myNext = soup.findAll('a', attrs = {'class':'nav next ui_button primary taLnk'})
    for s in list:
        tmp = BeautifulSoup(str(s),"lxml")
        tmpList = tmp.findAll('a', href=True)
        for element in tmpList:
            arr = str(element).split(' ')
            tmp_arr = str(arr[1]).split('=')
            url = str(tmp_arr[1])[2:-1]
            url_list.append(url)
        
url_list = set(url_list)
target_rating = []
count = 0;
review_url = []
for url in url_list:
    url = 'https://www.tripadvisor.com/' + url
    page = urllib2.urlopen(url).read()
    soup = BeautifulSoup(page, "lxml")
    innerBubble = soup.findAll('div', attrs = {'class':'reviewSelector'})
    tmp =  BeautifulSoup(str(innerBubble[0]), "lxml")
    tmp_page = tmp.findAll('div', attrs = {'class':'quote'})
    div = BeautifulSoup(str(tmp_page), "lxml")
    div_html = BeautifulSoup(str(div), "lxml")
    buff = div_html.findAll('a', href = True)
    #buff[0]['href']
    if(len(buff) == 1):
        review_url.append(buff[0]['href']);
        
for url in review_url:
    url = 'https://www.tripadvisor.com' + url
    page = urllib2.urlopen(url).read()
    soup = BeautifulSoup(page, "lxml")
    element = soup.findAll('div', attrs = {'class':'col rating '})
    element = BeautifulSoup(str(element), "lxml")
    temp = element.findAll('li')
    temp = BeautifulSoup(str(temp), "lxml")
    span = temp.findAll('span', attrs={'class': None})
    test = str(span).split('<span>')
    
    num1 = int(test[1].replace(",", ""))
    num2 = int(test[4].replace(",", ""))
    num3 = int(test[7].replace(",", ""))
    num4 = int(test[10].replace(",", ""))
    num5 = int(test[13].replace(",", ""))
    
    y = num1 * 5 + num2 + 4 + num3 * 3 + num4 * 2 + num5 * 1
    y = float(y) / (num1 + num2 + num3 + num4 + num5)
    y = round(y, 2)
    target_rating.append(y)

y_df = pd.DataFrame(target_rating)
y_df.to_csv('target.csv')  

df = pd.DataFrame(review_url)
df.to_csv('url.csv')

In [17]:
import urllib2
import csv
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

url = []
allVector = []
with open('url.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        arr = str(row).split('/')
        url.append(arr[1][:-2])
        
driver = webdriver.Firefox()
for element in url:
    value = 0
    location = 0
    s_quality = 0
    rooms = 0
    clean = 0
    service = 0
    total = 0
    c = 0
    link = 'https://www.tripadvisor.com/' + element
    page = urllib2.urlopen(link).read()
    soup = BeautifulSoup(page, "lxml")
    current = soup.findAll('span', attrs = {'class':'pageNum current'})
    my_next = soup.findAll('a', attrs = {'class':'nav next rndBtn ui_button primary taLnk'})
    innerBubble = soup.findAll('div', attrs = {'class':'innerBubble'})
    buff = BeautifulSoup(str(innerBubble), "lxml")
    recommend = buff.findAll('ul', attrs = {'class':'recommend'})
    
    for e in recommend:
        tmpSoup = BeautifulSoup(str(e), "lxml")
        tmpRec = tmpSoup.findAll('li', attrs = {'class':'recommend-answer'})
        if len(tmpRec) == 6:
            total = total + 1
            tmp0 = str(tmpRec[0])
            html0 = BeautifulSoup(tmp0, "lxml")
            current0 = html0.findAll('img', alt = True)
            value = value + int(current0[0].get('alt')[:1])
            
            tmp1 = str(tmpRec[1])
            html1 = BeautifulSoup(tmp1, "lxml")
            current1 = html1.findAll('img', alt = True)
            location = location + int(current1[0].get('alt')[:1])
            
            tmp2 = str(tmpRec[2])
            html2 = BeautifulSoup(tmp2, "lxml")
            current2 = html2.findAll('img', alt = True)
            s_quality = s_quality + int(current2[0].get('alt')[:1])
            
            tmp3 = str(tmpRec[3])
            html3 = BeautifulSoup(tmp3, "lxml")
            current3 = html3.findAll('img', alt = True)
            rooms = rooms + int(current3[0].get('alt')[:1])
            
            tmp4 = str(tmpRec[4])
            html4 = BeautifulSoup(tmp4, "lxml")
            current4 = html4.findAll('img', alt = True)
            clean = clean + int(current4[0].get('alt')[:1])
            
            tmp5 = str(tmpRec[5])
            html5 = BeautifulSoup(tmp5, "lxml")
            current5 = html5.findAll('img', alt = True)
            service = service + int(current5[0].get('alt')[:1])
    url_link = link
    html_page = urllib2.urlopen(url_link).read()
    bs = BeautifulSoup(html_page, "lxml")
    count = 0
    next_page = bs.findAll('a', attrs = {'class':'nav next rndBtn ui_button primary taLnk'})
    
    template = url_link[:-16] + 'or'
    html_page = urllib2.urlopen(url_link).read()
    bs = BeautifulSoup(html_page, "lxml")
    count = 0
    next_div = bs.findAll('a', attrs = {'class':'nav next rndBtn ui_button primary taLnk'})
    current = int(next_div[0].get('href')[-1:])
    while count < 200:
        url = template + str(current)
        driver.get(url)
    
        page_source = driver.page_source
        soup_2 = BeautifulSoup(page_source, "lxml")
        rating = soup_2.findAll('ul', attrs = {'class':'recommend'})
        for r in rating:
            ratingSoup = BeautifulSoup(str(r), "lxml")
            ratingRec = ratingSoup.findAll('li', attrs = {'class':'recommend-answer'})
            
            if len(ratingRec) == 6:
                total = total + 1
                rating_tmp0 = str(ratingRec[0])
                rating_html0 = BeautifulSoup(rating_tmp0, "lxml")
                rating_current0 = rating_html0.findAll('img', alt = True)
                value = value + int(rating_current0[0].get('alt')[:1])
                
                rating_tmp1 = str(ratingRec[1])
                rating_html1 = BeautifulSoup(rating_tmp1, "lxml")
                rating_current1 = rating_html1.findAll('img', alt = True)
                location = location + int(rating_current1[0].get('alt')[:1])
                
                rating_tmp2 = str(ratingRec[2])
                rating_html2 = BeautifulSoup(rating_tmp2, "lxml")
                rating_current2 = rating_html2.findAll('img', alt = True)
                s_quality = s_quality + int(rating_current2[0].get('alt')[:1])
                
                rating_tmp3 = str(ratingRec[3])
                rating_html3 = BeautifulSoup(rating_tmp3, "lxml")
                rating_current3 = rating_html3.findAll('img', alt = True)
                rooms = rooms + int(rating_current3[0].get('alt')[:1])
                
                rating_tmp4 = str(ratingRec[4])
                rating_html4 = BeautifulSoup(rating_tmp4, "lxml")
                rating_current4 = rating_html4.findAll('img', alt = True)
                clean = clean + int(rating_current4[0].get('alt')[:1])
                
                rating_tmp5 = str(ratingRec[5])
                rating_html5 = BeautifulSoup(rating_tmp5, "lxml")
                rating_current5 = rating_html5.findAll('img', alt = True)
                service = service + int(rating_current0[0].get('alt')[:1])
        current = current + 6
        count = count + 1
    vector = [value, location, s_quality, rooms, clean, service]
    #print vector
    #print total
    if(total == 0):
        v = [3, 3, 3, 3, 3, 3]
    else:
        vc = [float(x) / total for x in vector]
        v = [ round(elem, 3) for elem in vc ]
    
    print v
    allVector.append(v)
print allVector
print len(allVector)

data_x = pd.DataFrame(allVector)
data_x.to_csv('clean_data.csv')

[3.951, 4.872, 4.361, 4.205, 4.467, 3.954]
[4.029, 4.451, 4.403, 4.403, 4.61, 4.029]
[3.747, 4.747, 3.973, 3.853, 4.4, 3.747]
[4.054, 4.44, 4.021, 3.831, 4.443, 4.054]
[4.207, 4.872, 4.079, 3.879, 4.534, 4.207]
[3.0, 2.674, 3.196, 3.043, 3.478, 3.0]
[4.007, 4.688, 4.366, 4.359, 4.544, 4.01]
[3.838, 4.531, 4.494, 4.351, 4.649, 3.838]
[4.346, 4.843, 4.738, 4.738, 4.847, 4.346]
[3.805, 4.857, 4.405, 4.271, 4.471, 3.805]
[4.023, 4.761, 4.258, 4.453, 4.638, 4.023]
[4.113, 4.374, 4.247, 3.725, 4.341, 4.115]
[3.743, 4.515, 4.245, 4.238, 4.557, 3.743]
[3.794, 4.008, 4.293, 4.289, 4.63, 3.794]
[3.913, 4.614, 4.157, 4.003, 4.205, 3.913]
[3.572, 3.736, 4.049, 3.974, 4.15, 3.576]
[4.085, 4.786, 4.201, 4.147, 4.46, 4.085]
[4.521, 4.884, 4.664, 4.63, 4.925, 4.534]
[4.256, 4.911, 4.804, 4.686, 4.894, 4.256]
[4.401, 4.914, 4.58, 4.572, 4.781, 4.401]
[4.067, 3.351, 3.937, 4.138, 4.212, 4.074]
[4.003, 4.489, 4.383, 4.391, 4.574, 4.003]
[4.186, 4.834, 4.563, 4.523, 4.727, 4.186]
[4.12, 4.827, 4.363, 4.15

In [78]:
import urllib2
import csv
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup


url = []
allVector = []
with open('url.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        arr = str(row).split('/')
        url.append(arr[1][:-2])
        
travel_types = []
for element in url:
    url = 'https://www.tripadvisor.com/' + element
    page = urllib2.urlopen(url).read()
    soup = BeautifulSoup(page, "lxml")
    div = soup.findAll('div', attrs = {'class':'col segment '})
    travel_type = BeautifulSoup(str(div), "lxml")
    li = travel_type.findAll('li')
    vector = []
    for a in li:
        tmp = BeautifulSoup(str(a), "lxml")
        buff = tmp.findAll('span', attrs={'class': None})
        num = BeautifulSoup(str(buff[0])).find(text=True)
        vector.append(int(num[1:-1].replace(",", "")))
    travel_types.append(vector)

print len(travel_types)
df = pd.DataFrame(travel_types)
df.to_csv('travel_type.csv') 

81


In [42]:
#Extract omni-parker
#https://www.tripadvisor.com/ShowUserReviews-g60745-d89599-r359484392-Omni_Parker_House-Boston_Massachusetts.html#or7
import urllib2
import csv
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Firefox()
count = 0
current = 7
review_list = []
template = "https://www.tripadvisor.com/ShowUserReviews-g60745-d89599-r359484392-Omni_Parker_House-Boston_Massachusetts.html#or"
while count < 800:
    url = template + str(current)
    #print url
    driver.get(url)
    
    page_source = driver.page_source
    soup_2 = BeautifulSoup(page_source, "lxml")
    review_block = soup_2.findAll('div', attrs = {'class':'  reviewSelector '})
    review_block.pop(0)
    for r in review_block:
        reviewId = r.get('id')
        ratingSoup = BeautifulSoup(str(r), "lxml")
        ratingRec = ratingSoup.findAll('li', attrs = {'class':'recommend-answer'})
        for e in ratingRec:
            rating_tmp0 = str(e)
            rating_html0 = BeautifulSoup(rating_tmp0, "lxml")
            rating_current0 = rating_html0.findAll('img', alt = True)
            current0 = rating_html0.findAll('img', alt = True)
            type0 = rating_html0.findAll('div', attrs = {'class':'recommend-description'})
            value = int(current0[0].get('alt')[:1])
            value_tmp = str(value)
            st = reviewId + ":" + type0[0].string + ":" + value_tmp
            review_list.append(st)
    count = count + 1
    current = current + 6

f = open('output.txt', 'w')
for element in review_list:
    f.write(element+'\n')
f.close()

# README: 

I extracted data and saved them in CSV files instead of printing them out. Please take a look at the csv file in the Git repository. The 'target.csv' is the overall rating of each hotel. I extrated data of 'traveller rating' on the web page and aggregate them to caculate the overall rating of each hotel. The 'clean_data.csv' contains ratings of attributes(clealiness, location, etc) that we want to use for further analysis, and a vector contains value of those 6 attributes is representing a hotel.To get each of the attributes(clealiness, location, etc), I retrived almost half of all reviews of each hotel and sum them up and devide by total review number. So the rate for each atttribute is in the range of 1 to 5. The 'travel_type.csv' file contains data of travel type of each restaurant(families, couples, business, etc). 'Output.txt' is the file contains the reviews of Omin parker hotel.

** Task 2 (20 pts) **

Now, we will use regression to analyze this information. First, we will fit a linear regression model that predicts the average rating. For example, for the hotel above, the average rating is

$$ \text{AVG_SCORE} = \frac{1*31 + 2*33 + 3*98 + 4*504 + 5*1861}{2527}$$

Use the model to analyze the important factors that decide the $\text{AVG_SCORE}$.

In [18]:
import numpy as np
from sklearn import datasets, linear_model
import pandas as pd
import sklearn.cross_validation as cross_validation
import statsmodels.api as sm
#value, location, sleep quality, rooms, cleanliness, service

# Create linear regression object
regr = linear_model.LinearRegression()
y = pd.read_csv('target.csv')
x = pd.read_csv('clean_data.csv')
target = []
training_set = []

for index, row in y.iterrows():
    target.append(row[1])

target_y = [ round(elem, 2) for elem in target ]
#row 4
for index, row in x.iterrows():
    tmp = [row[1], row[2], row[3], row[4], row[5], row[6]]
    v = [ round(elem, 2) for elem in tmp ]
    training_set.append(v)
    
ax = np.array(training_set)
target_value = np.array(target_y)

model = sm.OLS(target_value, ax)
results = model.fit()
print results.summary()
print "Confidence Intervals:", results.conf_int()
print "Parameters:", results.params

X_train, X_test, y_train, y_test = cross_validation.train_test_split(ax, target_value, test_size=0.5, random_state=0)


regr.fit(X_train, y_train)
print('Coefficients: \n', regr.coef_)

# The mean square error
print("Residual sum of squares: %.2f"
      % np.mean((regr.predict(X_test) - y_test) ** 2))


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.985
Model:                            OLS   Adj. R-squared:                  0.984
Method:                 Least Squares   F-statistic:                     814.9
Date:                Tue, 29 Mar 2016   Prob (F-statistic):           3.88e-66
Time:                        23:50:42   Log-Likelihood:                -40.202
No. Observations:                  81   AIC:                             92.40
Df Residuals:                      75   BIC:                             106.8
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
x1            -4.2418     19.692     -0.215      0.8

# README:

The way I do it is to elimiate one attribute but keep the other 5 factors in the vector for each time. For example, I will delete 'value' attribute but keep the rest of 5 and see the mean square value. If I ignore 'value', 'clenliness', factors (only delete one for each time), the mean square error increases, this shows they are relatively important factors. And their coefficiencies are bigger than the rest.

** Task 3 (30 pts) **

Finally, we will use logistic regression to decide if a hotel is _excellent_ or not. We classify a hotel as _excellent_ if more than **60%** of its ratings are 5 stars. This is a binary attribute on which we can fit a logistic regression model. As before, use the model to analyze the data.

In [ ]:
import urllib2
import csv
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup


url = []
allVector = []
with open('url.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        arr = str(row).split('/')
        url.append(arr[1][:-2])
        
hotel_label = []
for link in url:
    url = 'https://www.tripadvisor.com/' + link
    page = urllib2.urlopen(url).read()
    soup = BeautifulSoup(page, "lxml")
    element = soup.findAll('div', attrs = {'class':'col rating '})
    element = BeautifulSoup(str(element), "lxml")
    temp = element.findAll('li')
    temp = BeautifulSoup(str(temp), "lxml")
    span = temp.findAll('span', attrs={'class': None})
    test = str(span).split('<span>')
    
    num1 = int(test[1].replace(",", ""))
    num2 = int(test[4].replace(",", ""))
    num3 = int(test[7].replace(",", ""))
    num4 = int(test[10].replace(",", ""))
    num5 = int(test[13].replace(",", ""))
    
    total = (num1 + num2 + num3 + num4 + num5)
    #print num1
    if (float(num1) / total) >= 0.60:
        hotel_label.append(1)
    else:
        hotel_label.append(0)
print hotel_label
df = pd.DataFrame(hotel_label)
df.to_csv('hotel_label.csv')  

In [6]:
import numpy as np
from sklearn import linear_model, datasets
import pandas as pd
import sklearn.cross_validation as cross_validation
import statsmodels.api as sm
#value, location, sleep quality, rooms, cleanliness, service

# Create linear regression object
logreg = linear_model.LogisticRegression(C=1e5)
y = pd.read_csv('hotel_label.csv')
x = pd.read_csv('clean_data.csv')
target = []
training_set = []

for index, row in y.iterrows():
    target.append(row[1])

target_y = [ round(elem, 2) for elem in target ]
#row 4
for index, row in x.iterrows():
    tmp = [row[1], row[2], row[3], row[4], row[5], row[6]]
    v = [ round(elem, 2) for elem in tmp ]
    training_set.append(v)
    
ax = np.array(training_set)
target_value = np.array(target_y)

logit = sm.Logit(target_value, ax)
 
# fit the model
result = logit.fit() 
print result.summary()

h = .02

X_train, X_test, y_train, y_test = cross_validation.train_test_split(ax, target_value, test_size=0.6, random_state=0)

#print a
# we create an instance of Neighbours Classifier and fit the data.
logreg.fit(X_train, y_train)

hypothesis = logreg.predict(X_test)

total = len(hypothesis)
count = 0
for f, b in zip(hypothesis, y_test):
    if f == b:
        count = count + 1

print float(count) / total

Optimization terminated successfully.
         Current function value: 0.547459
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                   81
Model:                          Logit   Df Residuals:                       75
Method:                           MLE   Df Model:                            5
Date:                Tue, 29 Mar 2016   Pseudo R-squ.:                 0.06393
Time:                        17:37:43   Log-Likelihood:                -44.344
converged:                       True   LL-Null:                       -47.373
                                        LLR p-value:                    0.3007
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
x1            67.2571     46.012      1.462      0.144       -22.924   157.439
x2            -0.3197      0.

# README:

by using logistic regression, my prediction has an 67% of accuracy

-------

In [ ]:
# Code for setting the style of the notebook
from IPython.core.display import HTML
def css_styling():
    styles = open("../../theme/custom.css", "r").read()
    return HTML(styles)
css_styling()